In [1]:
import tensorflow.compat.v1 as tf
import warnings
tf.disable_v2_behavior()
tf.set_random_seed(777)  # for reproducibility
warnings.filterwarnings('ignore')
print(tf.__version__)

Instructions for updating:
non-resource variables are not supported in the long term
2.4.0


In [2]:
import tensorflow as tf2 #Tensorflow 2.x
import numpy as np
import random

In [3]:
# hyper parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
mnist = tf.keras.datasets.mnist
nb_classes = 10

(x_train, y_train), (x_test, y_test) = mnist.load_data()
# normalizing data
x_train, x_test = x_train / 255.0, x_test / 255.0

# change data shape
print(x_train.shape)  # (60000, 28, 28)

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1] * x_train.shape[2])
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1] * x_test.shape[2])

y_train = tf.keras.utils.to_categorical(y_train, nb_classes)
y_test = tf.keras.utils.to_categorical(y_test, nb_classes)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000, 784)
(60000, 10)
(10000, 784)
(10000, 10)


In [5]:
# input place holders
X = tf.placeholder(tf.float32, [None, 784])
X_img = tf.reshape(X, [-1, 28, 28, 1])   # img 28x28x1 (black/white)

Y = tf.placeholder(tf.float32, [None, 10])

In [6]:
# L1 ImgIn shape=(?, 28, 28, 1)
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
#    Conv     -> (?, 28, 28, 32)
#    Pool     -> (?, 14, 14, 32)
L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], padding='SAME')

In [7]:
# L2 ImgIn shape=(?, 14, 14, 32)
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
#    Conv      ->(?, 14, 14, 64)
#    Pool      ->(?, 7, 7, 64)
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], padding='SAME')
L2_flat = tf.reshape(L2, [-1, 7 * 7 * 64])

In [8]:
init = tf2.initializers.GlorotUniform()
W3 = tf.get_variable("W3", shape=[7 * 7 * 64, 10],initializer=init)

b = tf.Variable(tf.random_normal([10]))
logits = tf.matmul(L2_flat, W3) + b

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


In [9]:
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [10]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [11]:
# train my model
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = 600

    for i in range(total_batch):
        batch_xs, batch_ys = next_batch(batch_size, x_train, y_train)
        feed_dict = {X: batch_xs, Y: batch_ys}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.
Epoch: 0001 cost = 0.356851613
Epoch: 0002 cost = 0.096391239
Epoch: 0003 cost = 0.066585898
Epoch: 0004 cost = 0.052306832
Epoch: 0005 cost = 0.044770456
Epoch: 0006 cost = 0.039048234
Epoch: 0007 cost = 0.034022930
Epoch: 0008 cost = 0.029934726
Epoch: 0009 cost = 0.027043472
Epoch: 0010 cost = 0.021594783
Epoch: 0011 cost = 0.022363485
Epoch: 0012 cost = 0.018026882
Epoch: 0013 cost = 0.015955185
Epoch: 0014 cost = 0.013673221
Epoch: 0015 cost = 0.013448320
Learning Finished!


In [14]:
# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(logits, axis=1), tf.argmax(Y, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(
    "Accuracy:",
    sess.run(accuracy, feed_dict={X: x_test, Y: y_test}),
)

# Get one and predict
r = random.randint(0, x_test.shape[0]-1)
real=sess.run(tf.argmax(y_test[r]))
predict=sess.run(tf.argmax(logits, axis=1), feed_dict={X: x_test[r : r + 1]})
print("Label: ",real)
print("Prediction: ",predict)


Accuracy: 0.9885
Label:  6
Prediction:  [6]
